<style>
    .info-card {
        max-width: 650px;
        margin: 25px auto;
        padding: 25px 30px;
        border: 1px solid #e0e0e0;
        border-radius: 12px;
        box-shadow: 0 4px 12px rgba(0, 0, 0, 0.05);
        font-family: -apple-system, BlinkMacSystemFont, "Segoe UI", Roboto, "Helvetica Neue", Arial, sans-serif;
        background-color: #fdfdfd;
        color: #333;
    }
    .info-card .title {
        color: #1a237e; /* Dark Indigo */
        font-size: 24px;
        font-weight: 600;
        margin-top: 0;
        margin-bottom: 15px;
        text-align: center;
        border-bottom: 2px solid #e8eaf6; /* Light Indigo */
        padding-bottom: 10px;
    }
    .info-card .details-grid {
        display: grid;
        grid-template-columns: max-content 1fr;
        gap: 12px 20px;
        margin-top: 20px;
        font-size: 16px;
    }
    .info-card .label {
        font-weight: 600;
        color: #555;
        text-align: right;
    }
    .info-card .value {
        font-weight: 400;
        color: #222;
    }
</style>

<div class="info-card">
    <h2 class="title">Unit 2 Exercise</h2>
    <div class="details-grid">
        <div class="label">Name:</div>
        <div class="value">Ethan Jed V. Carbonell</div>
        <div class="label">Date:</div>
        <div class="value">September 12, 2025</div>
        <div class="label">Year & Section:</div>
        <div class="value">BSCS 3A AI</div>
        <div></div>
    </div>
</div>

# Dense Layer Class

## Functions

### Constructor
Creates a new dense layer with specified number of inputs and neurons. Initializes random weights and zero biases if not provided.

- **Inputs:** Number of inputs, number of neurons, weights, biases  
- **Outputs:** New dense layer object

### forward
Performs the linear transformation by multiplying inputs with weights and adding biases.

- **Inputs:** Input data array  
- **Outputs:** Raw output before activation

### activate  
Applies non-linear activation function to the raw output.

- **Inputs:** Activation function name (relu, sigmoid, or softmax)  
- **Outputs:** Activated output

### calculate_categorical_loss
Computes cross-entropy loss for multi-class classification problems.

- **Inputs:** Predicted probabilities, true labels (one-hot encoded)  
- **Outputs:** Average loss value

### calculate_bce_loss
Computes binary cross-entropy loss for binary classification problems.

- **Inputs:** Predicted probabilities, true labels (zero or one)  
- **Outputs:** Average loss value

In [57]:
import numpy as np

class Dense_Layer:
    def __init__(self, n_inputs, n_neurons, weights=None, biases=None):
        if weights is None or biases is None:
            self.weights = 0.10 * np.random.randn(n_inputs, n_neurons)
            self.biases = np.zeros((1, n_neurons))
        else:
            self.weights = np.array(weights)
            self.biases = np.array(biases).reshape(1, -1)
            
        if self.weights.shape != (n_inputs, n_neurons):
            raise ValueError(f"Shape of weights {self.weights.shape} does not match expected shape {(n_inputs, n_neurons)}")
        if self.biases.shape != (1, n_neurons):
            raise ValueError(f"Shape of biases {self.biases.shape} does not match expected shape {(1, n_neurons)}")

    def forward(self, inputs):
        self.inputs = np.array(inputs)
        self.raw_output = np.dot(self.inputs, self.weights) + self.biases
        return self.raw_output

    def activate(self, activation_function):
        self.activation_name = activation_function.lower()
        
        if self.activation_name == 'relu':
            self.output = np.maximum(0, self.raw_output)
        elif self.activation_name == 'sigmoid':
            self.output = 1 / (1 + np.exp(-self.raw_output))
        elif self.activation_name == 'softmax':
            exp_values = np.exp(self.raw_output - np.max(self.raw_output, axis=1, keepdims=True))
            self.output = exp_values / np.sum(exp_values, axis=1, keepdims=True)
        else:
            raise ValueError("Unsupported activation function. Choose from 'relu', 'sigmoid', 'softmax'.")
        
        return self.output

    @staticmethod
    def calculate_categorical_loss(y_pred, y_true):
        y_pred_clipped = np.clip(y_pred, 1e-7, 1 - 1e-7)
        correct_confidences = np.sum(y_pred_clipped * y_true, axis=1)
        negative_log_likelihoods = -np.log(correct_confidences)
        return np.mean(negative_log_likelihoods)
        
    @staticmethod
    def calculate_bce_loss(y_pred, y_true):
        y_true_np = np.array(y_true)
        
        y_pred_clipped = np.clip(y_pred, 1e-7, 1 - 1e-7)
        loss = -(y_true_np * np.log(y_pred_clipped) + (1 - y_true_np) * np.log(1 - y_pred_clipped))
        return np.mean(loss)


# Problem 1: Iris Dataset Classification

> Given the following inputs from the Iris Dataset, using the sepal length, sepal width, petal length, and petal width, determine the class of the flower (Iris-setosa, Iris-versicolor, or Iris-virginica) by calculating the final output of the neural network.

---
## Input Data

| **Input Features ($X$)** | **Target Output ($Y_{target}$)** |
| :--- | :--- |
| <pre>X = [5.1]<br>    [3.5]<br>    [1.4]<br>    [0.2]</pre> | <pre>Y = [0.7]  # Iris-setosa<br>    [0.2]  # Iris-versicolor<br>    [0.1]  # Iris-virginica</pre> |

## Network Architecture

| **Component** | **First Hidden Layer** | **Second Hidden Layer** | **Output Layer** |
| :--- | :--- | :--- | :--- |
| **Weights ($W$)** | <pre>W₁ = [ 0.2,  0.5, -0.3]<br>     [ 0.1, -0.2,  0.4]<br>     [-0.4,  0.3,  0.2]<br>     [ 0.6, -0.1,  0.5]</pre> | <pre>W₂ = [ 0.3, -0.5]<br>     [ 0.7,  0.2]<br>     [-0.6,  0.4]</pre> | <pre>W₃ = [ 0.5, -0.3,  0.8]<br>     [-0.2,  0.6, -0.4]</pre> |
| **Bias ($B$)** | <pre>B₁ = [ 3.0]<br>     [-2.1]<br>     [ 0.6]</pre> | <pre>B₂ = [4.3]<br>     [6.4]</pre> | <pre>B₃ = [-1.5]<br>     [ 2.1]<br>     [-3.3]</pre> |
| **Activation** | `ReLU` | `Sigmoid` | `Softmax` |

In [54]:
if __name__ == "__main__":
    print("Iris Dataset Neural Network\n")
    inputs = [[5.1, 3.5, 1.4, 0.2]]
    
    target_output = [[0.7, 0.2, 0.1]]

    weights1 = [
        [0.2, 0.5, -0.3],
        [0.1, -0.2, 0.4],
        [-0.4, 0.3, 0.2],
        [0.6, -0.1, 0.5]
    ]
    biases1 = [3.0, -2.1, 0.6]

    weights2 = [
        [0.3, -0.5],
        [0.7, 0.2],
        [-0.6, 0.4]
    ]
    biases2 = [4.3, 6.4]

    weights3 = [
        [0.5, -0.3, 0.8],
        [-0.2, 0.6, -0.4]
    ]
    biases3 = [-1.5, 2.1, -3.3]
    
    print(f"Input data: {np.array(inputs)}")
    print(f"Target output: {np.array(target_output)}\n")

    layer1 = Dense_Layer(n_inputs=4, n_neurons=3, weights=weights1, biases=biases1)
    layer2 = Dense_Layer(n_inputs=3, n_neurons=2, weights=weights2, biases=biases2)
    layer3 = Dense_Layer(n_inputs=2, n_neurons=3, weights=weights3, biases=biases3)

    
    print("Layer 1 [ReLU]  ")
    layer1.forward(inputs)
    layer1_output = layer1.activate('relu')
    print(f"Output: {layer1_output}\n")
    
    print("Layer 2 [Sigmoid]  ")
    layer2.forward(layer1_output)
    layer2_output = layer2.activate('sigmoid')
    print(f"Output: {layer2_output}\n")

    print("Layer 3 [Softmax]  ")
    layer3.forward(layer2_output)
    final_output = layer3.activate('softmax')
    print(f"Output: {final_output}\n")

        
    print("\n----------------------------------------------------------------------------\nFINAL RESULTS\n")
    print(f"Final Predicted Output (Probabilities): {final_output}")
    
    iris_species = ['Iris-setosa', 'Iris-versicolor', 'Iris-virginica']
    predicted_class_index = np.argmax(final_output)
    predicted_species = iris_species[predicted_class_index]
    
    print(f"Predicted Iris Species: {predicted_species}\n")
    
    loss = Dense_Layer.calculate_categorical_loss(final_output, target_output)
    print(f"Loss (Categorical Cross-Entropy): {loss:.4f}")

Iris Dataset Neural Network

Input data: [[5.1 3.5 1.4 0.2]]
Target output: [[0.7 0.2 0.1]]

Layer 1 [ReLU]  
Output: [[3.93 0.15 0.85]]

Layer 2 [Sigmoid]  
Output: [[0.99378157 0.99187781]]

Layer 3 [Softmax]  
Output: [[0.0265075  0.96865119 0.00484132]]


----------------------------------------------------------------------------
FINAL RESULTS

Final Predicted Output (Probabilities): [[0.0265075  0.96865119 0.00484132]]
Predicted Iris Species: Iris-versicolor

Loss (Categorical Cross-Entropy): 1.5475


# Problem 2: Breast Cancer Classification

> Given the following inputs from the Breast Cancer Dataset, using three features (Mean Radius, Mean Texture, and Mean Smoothness), determine whether the tumor is **Benign (0)** or **Malignant (1)** by calculating the network's output step-by-step.

---
## Input Data

| **Input Features ($X$)** | **Target Output ($Y_{target}$)** |
| :--- | :--- |
| <pre>X = [14.1]   # Mean Radius<br>    [20.3]   # Mean Texture<br>    [0.095]  # Mean Smoothness</pre> | <pre>Y = [1]  # Malignant</pre> |


## Network Architecture

| **Component** | **First Hidden Layer** | **Second Hidden Layer** | **Output Layer** |
| :--- | :--- | :--- | :--- |
| **Weights ($W$)** | <pre>W₁ = [ 0.5, -0.3,  0.8]<br>     [-0.2,  0.4, -0.6]<br>     [-0.7,  0.9,  0.1]</pre> | <pre>W₂ = [ 0.6, -0.2,  0.4]<br>     [-0.3,  0.5,  0.7]</pre> | <pre>W₃ = [0.7, -0.5]</pre> |
| **Bias ($B$)** | <pre>B₁ = [ 0.3]<br>     [-0.5]<br>     [ 0.6]</pre> | <pre>B₂ = [ 0.1]<br>     [-0.8]</pre> | <pre>B₃ = [0.2]</pre> |
| **Activation** | `ReLU` | `Sigmoid` | `Sigmoid` |

In [55]:
if __name__ == "__main__":
    print("Breast Cancer Dataset Neural Network\n")

    inputs = [[14.1, 20.3, 0.095]]
    
    #1 for Malignant
    target_output = [[1]]

    weights1 = [
        [0.5, -0.3, 0.8],
        [-0.2, 0.4, -0.6],
        [-0.7, 0.9, 0.1]
    ]
    biases1 = [0.3, -0.5, 0.6]

    weights2_raw = [
        [0.6, -0.2, 0.4],
        [-0.3, 0.5, 0.7]
    ]
    weights2 = np.array(weights2_raw).T
    biases2 = [0.1, -0.8]

    weights3_raw = [[0.7, -0.5]]
    weights3 = np.array(weights3_raw).T
    biases3 = [0.2]
    
    print(f"Input data: {np.array(inputs)}")
    print(f"Target output: {target_output[0][0]} (Malignant)\n")

    layer1 = Dense_Layer(n_inputs=3, n_neurons=3, weights=weights1, biases=biases1)
    layer2 = Dense_Layer(n_inputs=3, n_neurons=2, weights=weights2, biases=biases2)
    layer3 = Dense_Layer(n_inputs=2, n_neurons=1, weights=weights3, biases=biases3)

    
    print("Layer 1 [ReLU]  ")
    layer1.forward(inputs)
    layer1_output = layer1.activate('relu')
    print(f"Output: {layer1_output}\n")
    
    print("Layer 2 [Sigmoid]  ")
    layer2.forward(layer1_output)
    layer2_output = layer2.activate('sigmoid')
    print(f"Output: {layer2_output}\n")

    print("Layer 3 [Sigmoid]  ")
    layer3.forward(layer2_output)
    final_output = layer3.activate('sigmoid')
    print(f"Output: {final_output}\n")
    
    
    print("\n----------------------------------------------------------------------------\nFINAL RESULTS\n")
    print(f"Final Predicted Output (Probability): {final_output[0][0]:.4f}")
    
    predicted_class = 1 if final_output[0][0] > 0.5 else 0
    prediction_label = "Malignant" if predicted_class == 1 else "Benign"
    
    print(f"Predicted Class: {predicted_class} ({prediction_label})\n")
    
    loss = Dense_Layer.calculate_bce_loss(final_output, target_output)
    print(f"Loss (Binary Cross-Entropy): {loss:.4f}")

Breast Cancer Dataset Neural Network

Input data: [[14.1   20.3    0.095]]
Target output: 1 (Malignant)

Layer 1 [ReLU]  
Output: [[3.2235 3.4755 0.    ]]

Layer 2 [Sigmoid]  
Output: [[0.79232544 0.49267552]]

Layer 3 [Sigmoid]  
Output: [[0.62440554]]


----------------------------------------------------------------------------
FINAL RESULTS

Final Predicted Output (Probability): 0.6244
Predicted Class: 1 (Malignant)

Loss (Binary Cross-Entropy): 0.4710
